In [1]:
import pandas as pd
import os
import numpy as np
from IPython.display import display, HTML, Markdown

## Hydrology API code

In [29]:
from enum import Enum
from concurrent.futures import ThreadPoolExecutor
from time import sleep
from io import StringIO
import urllib3
import ujson as json

class Measure(Enum):
    LEVEL = 'level'
    FLOW = 'flow'
    RAINFALL = 'rainfall'

class HydrologyApi:
    API_BASE_URL = "https://environment.data.gov.uk/hydrology/"
    DATA_DIR = "data"
    CACHE_DIR = "cache"
    
    units = {
        Measure.LEVEL: 'i-900-m-qualified',
        Measure.FLOW: 'i-900-m3s-qualified',
        Measure.RAINFALL: 't-900-mm-qualified',
    }
    
    observed_property_names = {
        Measure.LEVEL: 'waterLevel',
        Measure.FLOW: 'waterFlow',
        Measure.RAINFALL: 'rainfall',
    }
    
    def __init__(self, max_threads):
        self.http = urllib3.PoolManager(maxsize=max_threads)
        self.thread_pool = ThreadPoolExecutor(max_workers=max_threads)
    
    def get_stations_on_river(self, river: str, measure: Measure, limit=1000):
        api_url = self.API_BASE_URL + 'id/stations'
        result = self.http.request(
            'GET',
            api_url,
            fields={
                'observedProperty': HydrologyApi.observed_property_names[measure],
                'riverName': river,
                '_limit': limit,
                "status.label": "Active"
            }
        ).data.decode('utf-8')
        data = json.loads(result)
        return pd.DataFrame(data['items'])
        
    def get_stations_close_to_with_measure(self, lat, lon, radius, measure: Measure, limit=100):
        api_url = self.API_BASE_URL + 'id/stations'
        
        result = self.http.request(
            'GET',
            api_url,
            fields={
                'observedProperty': measure.value,
                'lat': lat,
                'long': lon,
                'dist': radius,
                'status.label':'Active',
                '_limit': limit
            }
        ).data.decode('utf-8')
        data = json.loads(result)
        return pd.DataFrame(data['items'])
    
    def get_measure(self, measure: Measure, station_id: str, start=None):
        api_url = self.API_BASE_URL + f"id/measures/{station_id}-{measure.value}-{HydrologyApi.units[measure]}/readings"
        # result = urlopen(api_url).read().decode('utf-8')
        result = self.http.request(
            'GET',
            api_url,
            fields={}
                | ({
                    'mineq-date': start.strftime('%Y-%m-%d')
                } if start is not None else {}),
        ).data.decode('utf-8')
        data = json.loads(result)
        return pd.DataFrame(data['items'])
    
    def _cached_batch_request(self, api_url):
        cache_key = api_url.split('?')[1]
        cache_key = cache_key.replace('=', '_').replace('&', '_')
        filepath = os.path.join(self.CACHE_DIR, f"{cache_key}.feather")
        
        if os.path.exists(filepath):
            print(f"Loading from cache: {api_url}")
            
            with open(filepath, 'rb') as f:
                return pd.read_feather(f)
            
        data = self._batch_request(api_url)
        
        if data is not None:
            if not os.path.exists(self.CACHE_DIR):
                os.makedirs(self.CACHE_DIR)
                
            data.to_feather(filepath)
            
        return data
    
    def _batch_request(self, api_url):
        status = "Pending"

        while status in ("Pending", "InProgress"):
            print(f"Making request to: {api_url}")
            
            request = self.http.request(
                'GET', 
                api_url, 
                headers={
                    'Accept-Encoding': 'gzip'
                }
            )
            content_type = request.headers['Content-Type']

            if content_type == 'text/csv':
                if len(request.data) == 0:
                    print('Got empty CSV')
                    return None
                buffer = StringIO(request.data.decode('utf-8'))
                return pd.read_csv(buffer, low_memory=False)
            
            assert content_type in (
                'application/json',
                'application/json;charset=UTF-8'), f"Unexpected content type: {content_type}"

            data = json.loads(request.data.decode('utf-8'))
            status = data["status"]

            if status == "Pending":
                print(f"Query is pending")
                pos_in_queue = data["positionInQueue"]
                print(f"Position in queue: {pos_in_queue}")
                eta = data["eta"] / 1000
                print(f"Estimated completion: {eta}")
                sleep(eta * 1.1)

            elif status == "InProgress":
                print(f"Query in progress")
                eta = data["eta"] / 1000
                print(f"Estimated completion: {eta}")
                sleep(eta * 1.1)

            elif status in ("Complete", "Completed"):
                print(f"Query completed: {data}")
                csv_url = data["dataUrl"] if "dataUrl" in data else data["url"]
                return pd.read_csv(csv_url)

            elif status == "Failed":
                raise Exception(f"Query failed, response: {data}")

            else:
                raise Exception(f"Unknown status: {data['status']}")
    
    def batch_get_measure(self, measure: Measure, station_id):
        try:
            api_url = self.API_BASE_URL + \
                f"data/batch-readings/batch/?measure={station_id}-{measure.value}-{HydrologyApi.units[measure]}&mineq-date=2007-01-01"
                
            return self._cached_batch_request(api_url)
        except Exception as e:
            print(f"Failed to get data for station: {station_id}, {e}")
            return None
        
    def batch_get_measure_on_river(self, measure: Measure, river):
        stations = self.get_stations_on_river(river)
        return self.batch_get_measure_from_stations(measure, stations)
        
    def batch_get_measure_from_stations(self, measure: Measure, stations):
        data = pd.DataFrame()
        threads = [
            self.thread_pool.submit(
                self.batch_get_measure, measure, station_id)
            for station_id in stations['notation'].values
        ]
        
        for thread, station_name in zip(threads, stations['label'].values):
            new_data = thread.result()
            if new_data is None:
                print(f"No new data for station: {station_name}")
                continue
            new_data = new_data.drop(columns=['measure', 'date', 'qcode', 'completeness'])
            new_data['station'] = station_name
            new_data['station'] = new_data['station'].astype('category')
            new_data['dateTime'] = pd.to_datetime(new_data['dateTime'])
            new_data['value'] = new_data['value'].astype(float)
            new_data['quality'] = new_data['quality'].astype('category')
            data = pd.concat([data, new_data])
            data.drop_duplicates(subset=['dateTime', 'station'], inplace=True)
        return data
        
        
    def get_filename(self, measure: Measure, river):
        return f"{river.lower().replace(' ', '_')}_{measure.value}_raw.feather"
    
    def update_dataframe(self, df: pd.DataFrame, measure: Measure, stations: pd.DataFrame):
        for station_name, station_id in stations[['label', 'notation']].values:
            
            last = df[df['station'] == station_name]['dateTime'].max() if len(df) > 0 else None
            
            new_measurements = self.get_measure(measure, station_id, last)[['dateTime', 'value', 'quality']]
            
            new_measurements['station'] = station_name
            new_measurements['station'] = new_measurements['station'].astype('category')
            new_measurements['dateTime'] = pd.to_datetime(new_measurements['dateTime'])
            new_measurements['value'] = new_measurements['value'].astype(float)
            
            df = pd.concat([df, new_measurements])
            
        df.drop_duplicates(subset=['dateTime', 'station'], inplace=True)
        return df
            
    def load(self, measure: Measure, stations: pd.DataFrame):
        df = self.batch_get_measure_from_stations(measure, stations)
        df = self.update_dataframe(df, measure, stations)
        return df        
        
def process_hydrology_data(df):
    return df[df['quality'].isin(['Good', 'Unchecked', 'Estimated'])] \
        .pivot(index='dateTime', columns='station', values='value') \
        .resample('15min').interpolate('time', limit_direction='both', limit=24*4, fill_value='extrapolate') \
        .astype(np.float16)

api = HydrologyApi(max_threads = 5)


## Download Level Data

In [30]:
stations = api.get_stations_on_river('River Wear', Measure.LEVEL)
print(f"Loading data for {len(stations)} stations: {stations['label'].values}")
level_df = api.load(Measure.LEVEL, stations)
level_df = process_hydrology_data(level_df)
level_df.info()

Loading data for 5 stations: ['Chester Le Street' 'Witton Park' 'Sunderland Bridge' 'Stanhope'
 'Durham New Elvet Bridge']
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=e7d8bbb6-5bba-4057-9f49-a299482c3348-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=05784319-693a-4d75-b29e-32f01a99ee4f-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=ddedb4d9-b2be-47c1-998d-acbc0ffb124b-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=b29c481a-5012-40f5-bb0c-f9370be34975-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=ba3f8598-e654-430d-9bb8-e1652e6ff93d-level-i-900-m-qualified

## Download Flow Data

In [31]:
stations = api.get_stations_on_river('River Wear', Measure.FLOW)
print(f"Loading data for {len(stations)} stations: {stations['label'].values}")
flow_df = api.load(Measure.FLOW, stations)
flow_df = process_hydrology_data(flow_df)
flow_df.info()

Loading data for 4 stations: ['Chester Le Street' 'Witton Park' 'Sunderland Bridge' 'Stanhope']
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=05784319-693a-4d75-b29e-32f01a99ee4f-flow-i-900-m3s-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=e7d8bbb6-5bba-4057-9f49-a299482c3348-flow-i-900-m3s-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=ddedb4d9-b2be-47c1-998d-acbc0ffb124b-flow-i-900-m3s-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=b29c481a-5012-40f5-bb0c-f9370be34975-flow-i-900-m3s-qualified&mineq-date=2007-01-01
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 588505 entries, 2007-01-01 00:00:00 to 2023-10-14 06:00:00
Freq: 15T
Data columns (total 4 columns):
 #   Column             No

We don't have flow data at New Elvet unfortunatly

## Download Rainfall Data

In [32]:
rainfall_stations = api.get_stations_close_to_with_measure(54.66305556, -1.67611111, 15, Measure.RAINFALL, limit=10)
bad_stations = ['15202aee-c5fd-404d-9de9-7357174ad10c']
rainfall_stations = rainfall_stations[~rainfall_stations['notation'].isin(bad_stations)].head(5)
print(f"Using {len(rainfall_stations)} rainfall stations: {rainfall_stations['label'].values}")

rainfall_df = api.load(Measure.RAINFALL, rainfall_stations)
rainfall_df = process_hydrology_data(rainfall_df)
rainfall_df.info()

Using 5 rainfall stations: ['Evenwood Gate' 'Harpington Hill Farm' 'Copley' 'Tunstall'
 'Darlington Lingfield Way']
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=1dabd12c-1d2e-4765-ae38-a4d5a121928d-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=bc34e640-d9ae-4362-8804-25d66ca66e4d-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=bf61ce31-b20e-4593-85dc-a083133b12ce-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=051f1b2a-6aca-4402-8956-5474ad39b12a-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=a8773476-0fde-40c7-a66b-0901e528e8f2-rainfall-t-900

In [38]:
level_df = level_df.fillna(level_df.mean())
flow_df = flow_df.fillna(flow_df.mean())
rainfall_df = rainfall_df.fillna(0)

d:\miniconda3\envs\river-level\Lib\site-packages\pandas\core\nanops.py:1488: RuntimeWarning: overflow encountered in cast
  return count.astype(dtype, copy=False)
d:\miniconda3\envs\river-level\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [39]:
df = pd.merge(
    level_df.add_prefix('Level '),
    flow_df.add_prefix('Flow '),
    left_index=True,
    right_index=True,
    how='outer',
)
df = pd.merge(
    df,
    rainfall_df.add_prefix('Rainfall '),
    left_index=True,
    right_index=True,
    how='outer',
)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 590967 entries, 2007-01-01 00:00:00 to 2023-11-08 21:30:00
Freq: 15T
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Level Chester Le Street            590967 non-null  float16
 1   Level Durham New Elvet Bridge      563826 non-null  float16
 2   Level Stanhope                     501615 non-null  float16
 3   Level Sunderland Bridge            589418 non-null  float16
 4   Level Witton Park                  585167 non-null  float16
 5   Flow Chester Le Street             588505 non-null  float16
 6   Flow Stanhope                      499151 non-null  float16
 7   Flow Sunderland Bridge             586956 non-null  float16
 8   Flow Witton Park                   582705 non-null  float16
 9   Rainfall Copley                    590967 non-null  float16
 10  Rainfall Darlington Lingfield Way  590967 non-null  float16


## Add Lag Features
for some reason makes loads of nans

In [49]:
df_rainfall_hourly = df.filter(regex='Rainfall').resample('1h').sum()
df_rainfall_six_hourly = df.filter(regex='Rainfall').resample('6h').sum()
df_rainfall_daily = df.filter(regex='Rainfall').resample('1d').sum()

target_cols = ['Level Durham New Elvet Bridge']
target_shifts = [-15, -30, -60, -90, -120]
level_shifts = [15, 30, 60, 90, 120]
flow_shifts = [15, 30, 60, 90, 120]
rainfall_min_shifts = [15, 30, 60]
rainfall_hour_shifts = [2, 3, 4, 5, 6]
rainfall_six_hour_shifts = [12, 18, 24, 30, 36, 42, 48]
rainfall_day_shifts = [3, 4, 5, 6, 7]

output_target_cols = []

df_lagged = df.copy()

for shift in target_shifts:
    shifted_df = df[target_cols].shift(shift, freq='min')
    shifted_df = shifted_df.add_suffix(f' {-shift:+d}min')
    output_target_cols.extend(shifted_df.columns)
    df_lagged = pd.concat([df_lagged, shifted_df], axis=1)

for shift in level_shifts:
    shifted_df = df.filter(regex='Level').shift(shift, freq='min')
    df_lagged = pd.concat([df_lagged, shifted_df.add_suffix(f' {-shift:+d}min')], axis=1)
    
for shift in flow_shifts:
    shifted_df = df.filter(regex='Flow').shift(shift, freq='min')
    df_lagged = pd.concat([df_lagged, shifted_df.add_suffix(f' {-shift:+d}min')], axis=1)

for shift in rainfall_min_shifts:
    shifted_df = df.filter(regex='Rainfall').shift(shift, freq='min')
    df_lagged = pd.concat([df_lagged, shifted_df.add_suffix(f' {-shift:+d}min')], axis=1)
    
for shift in rainfall_hour_shifts:
    shifted_df = df_rainfall_hourly.shift(shift, freq='h')
    df_lagged = pd.concat([df_lagged, shifted_df.add_suffix(f' {-shift:+d}h')], axis=1)
    
for shift in rainfall_six_hour_shifts:
    shifted_df = df_rainfall_six_hourly.shift(shift, freq='h')
    df_lagged = pd.concat([df_lagged, shifted_df.add_suffix(f' {-shift:+d}h')], axis=1)
    
for shift in rainfall_day_shifts:
    shifted_df = df_rainfall_daily.shift(shift, freq='d')
    df_lagged = pd.concat([df_lagged, shifted_df.add_suffix(f' {-shift:+d}d')], axis=1)
    
df_lagged = df_lagged.astype(np.float16)

In [ ]:
df_lagged.to_feather('data/river_wear_lagged.feather')

d:\miniconda3\envs\river-level\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
